<a href="https://colab.research.google.com/github/BhavyaaaJain/HSCode/blob/main/HSCode_optimized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow_text
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
import spacy

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [3]:
df = pd.read_excel("data.xlsx")

In [4]:
df

,Chapter Number,TariffItem,Description of goods,IGST,Unit,Unnamed: 5
0,CHAPTER 1,1012100,"Live Horses, Asses, Mules and Hinnies Horses:...",0.12,u,NaN
1,CHAPTER 1,10129,"Live Horses, Asses, Mules and Hinnies Horses:...",NaN,NaN,NaN
2,CHAPTER 1,1012910,"Live Horses, Asses, Mules and Hinnies Horses:...",0.12,u,NaN
3,CHAPTER 1,1012990,"Live Horses, Asses, Mules and Hinnies Horses:...",0.12,u,NaN
4,CHAPTER 1,10130,"Live Horses, Asses, Mules and Hinnies Asses:",NaN,NaN,NaN
...,...,...,...,...,...,...
13062,CHAPTER 97,97052900,Collections and collectors' pieces of archaeol...,0.12,u,NaN
13063,CHAPTER 97,97053100,Collections and collectors' pieces of archaeol...,0.12,u,NaN
13064,CHAPTER 97,97053900,Collections and collectors' pieces of archaeol...,0.12,u,NaN
13065,CHAPTER 97,97061000,Antiques of an age exceeding 100 years Of an ...,0.12,u,NaN


In [5]:
df=df.drop(['IGST', 'Unit', 'Unnamed: 5'], axis=1)
df

,Chapter Number,TariffItem,Description of goods
0,CHAPTER 1,1012100,"Live Horses, Asses, Mules and Hinnies Horses:..."
1,CHAPTER 1,10129,"Live Horses, Asses, Mules and Hinnies Horses:..."
2,CHAPTER 1,1012910,"Live Horses, Asses, Mules and Hinnies Horses:..."
3,CHAPTER 1,1012990,"Live Horses, Asses, Mules and Hinnies Horses:..."
4,CHAPTER 1,10130,"Live Horses, Asses, Mules and Hinnies Asses:"
...,...,...,...
13062,CHAPTER 97,97052900,Collections and collectors' pieces of archaeol...
13063,CHAPTER 97,97053100,Collections and collectors' pieces of archaeol...
13064,CHAPTER 97,97053900,Collections and collectors' pieces of archaeol...
13065,CHAPTER 97,97061000,Antiques of an age exceeding 100 years Of an ...


In [6]:
df.columns=['chapter', 'code', 'desc']
df

,chapter,code,desc
0,CHAPTER 1,1012100,"Live Horses, Asses, Mules and Hinnies Horses:..."
1,CHAPTER 1,10129,"Live Horses, Asses, Mules and Hinnies Horses:..."
2,CHAPTER 1,1012910,"Live Horses, Asses, Mules and Hinnies Horses:..."
3,CHAPTER 1,1012990,"Live Horses, Asses, Mules and Hinnies Horses:..."
4,CHAPTER 1,10130,"Live Horses, Asses, Mules and Hinnies Asses:"
...,...,...,...
13062,CHAPTER 97,97052900,Collections and collectors' pieces of archaeol...
13063,CHAPTER 97,97053100,Collections and collectors' pieces of archaeol...
13064,CHAPTER 97,97053900,Collections and collectors' pieces of archaeol...
13065,CHAPTER 97,97061000,Antiques of an age exceeding 100 years Of an ...


In [7]:
#give ids to each chapter
chapter_mapping = {i: chapter for i, chapter in enumerate(df['chapter'].unique())}

In [8]:
#remove characters and replace digits with hash
pattern = r'[^a-zA-Z0-9\s]'
df['desc'] = df['desc'].str.replace(pattern, " ", regex=True)
pattern = r'[\d]'
df['desc'] = df['desc'].str.replace(pattern, '#', regex=True)
df

,chapter,code,desc
0,CHAPTER 1,1012100,Live Horses Asses Mules and Hinnies Horses ...
1,CHAPTER 1,10129,Live Horses Asses Mules and Hinnies Horses ...
2,CHAPTER 1,1012910,Live Horses Asses Mules and Hinnies Horses ...
3,CHAPTER 1,1012990,Live Horses Asses Mules and Hinnies Horses ...
4,CHAPTER 1,10130,Live Horses Asses Mules and Hinnies Asses
...,...,...,...
13062,CHAPTER 97,97052900,Collections and collectors pieces of archaeol...
13063,CHAPTER 97,97053100,Collections and collectors pieces of archaeol...
13064,CHAPTER 97,97053900,Collections and collectors pieces of archaeol...
13065,CHAPTER 97,97061000,Antiques of an age exceeding ### years Of an ...


In [9]:
#remove stop words
def remove_stopwords(text):
    words = nltk.word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stopwords.words('english')]
    return ' '.join(filtered_words)

In [10]:
df['desc'] = df['desc'].apply(remove_stopwords)

In [11]:
df.head()

,chapter,code,desc
0,CHAPTER 1,1012100,Live Horses Asses Mules Hinnies Horses Pure br...
1,CHAPTER 1,10129,Live Horses Asses Mules Hinnies Horses
2,CHAPTER 1,1012910,Live Horses Asses Mules Hinnies Horses Horses ...
3,CHAPTER 1,1012990,Live Horses Asses Mules Hinnies Horses
4,CHAPTER 1,10130,Live Horses Asses Mules Hinnies Asses


In [12]:
# mapping the ids to the common keywords among each chapters using TF-IDF
chapters = {}
for chapter_id, chapter in chapter_mapping.items():
    chapter_data = df[df['chapter'] == chapter]
    chapter_descriptions = chapter_data['desc']

    # TF-IDF vectorization for the current chapter
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(chapter_descriptions)

    # Get feature names (words)
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # Calculate TF-IDF scores
    tfidf_scores = tfidf_matrix.sum(axis=0).A1

    # Sort words by TF-IDF score and select the top 50 as keywords
    num_keywords = 50
    top_keywords_indices = tfidf_scores.argsort()[-num_keywords:][::-1]
    top_keywords = [feature_names[i] for i in top_keywords_indices]

    # Create a mapping of ids to keywords of a particular chapter
    # Store the keyword mapping in the chapter_keyword_mapping dictionary
    chapters[chapter_id] = top_keywords

In [13]:
df.head()

,chapter,code,desc
0,CHAPTER 1,1012100,Live Horses Asses Mules Hinnies Horses Pure br...
1,CHAPTER 1,10129,Live Horses Asses Mules Hinnies Horses
2,CHAPTER 1,1012910,Live Horses Asses Mules Hinnies Horses Horses ...
3,CHAPTER 1,1012990,Live Horses Asses Mules Hinnies Horses
4,CHAPTER 1,10130,Live Horses Asses Mules Hinnies Asses


In [14]:
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1) + len(set2) - intersection
    return intersection / union

In [15]:
def match_to_chapter(user_input):
    best_match = []
    best_similarity = 0

    # Tokenize the user input
    input_keywords = set(user_input.split())

    for chapter_id, chapter_keywords in chapters.items():
        # Calculate Jaccard similarity between input keywords and chapter keywords
        similarity = jaccard_similarity(input_keywords, chapter_keywords)
        # print(chapter_id, similarity)
        # Check if the similarity exceeds the threshold
        if similarity >= best_similarity and similarity!=0:
            best_match.append(chapter_id)
            best_similarity = similarity

    matched_chapter = best_match
    if len(matched_chapter)==0:
      for chapter_id, chapter in chapter_mapping.items():
        matched_chapter.append(chapter_id)

    return matched_chapter

In [17]:
# def encode_text_matched(matched_chapter):
#   matched_embeddings={}
#   for ch in matched_chapter:
#     chapter_data = df[df['chapter']==chapter_mapping[ch]]
#     for ind, row in chapter_data.iterrows():
#       text = row['desc']
#       preprocessed_text = bert_preprocess([text])
#       bert_encoded = bert_encoder(preprocessed_text)['pooled_output']
#       matched_embeddings[row['code']] = bert_encoded
#   return matched_embeddings

In [27]:
def encode_text_matched(matched_chapter, batch_size):
  matched_embeddings={}
  # Initialize an empty list to collect batches of text
  batch_text = []
  batch_codes = []
  for ch in matched_chapter:
    chapter_data = df[df['chapter']==chapter_mapping[ch]]
    for ind, row in chapter_data.iterrows():
      text = row['desc']
      batch_text.append(text)
      batch_codes.append(row['code'])
    if len(batch_text) >= batch_size:
      # Encode the batch of text in one go
      preprocessed_text = bert_preprocess(batch_text)
      bert_encoded = bert_encoder(preprocessed_text)['pooled_output']

      # Update the matched_embeddings dictionary with batch results
      for code, embedding in zip(batch_codes, bert_encoded):
          matched_embeddings[code] = embedding.numpy()

      batch_text = []
      batch_codes = []

  return matched_embeddings


      # text = row['desc']
      # preprocessed_text = bert_preprocess([text])
      # bert_encoded = bert_encoder(preprocessed_text)['pooled_output']
      # matched_embeddings[row['code']] = bert_encoded
  # return matched_embeddings

In [19]:
#encode the input
def encode_input(input):
  preprocessed_text = bert_preprocess([input])
  bert_encoded = bert_encoder(preprocessed_text)['pooled_output']
  return bert_encoded

In [20]:
#find the cosine similarity between input and each embeddings stored

def find_similar(matched_embeddings, input_encoded):
  similarities={}
  for code, embeddings in matched_embeddings.items():
    embeddings = embeddings.reshape(1, -1)
    # print(embeddings.shape)
    similarity_score = cosine_similarity(input_encoded, embeddings)[0][0]
    chapter_data = df[df['code']==code]
    text = chapter_data['desc'].values[0]
    similarities[code]=[text, similarity_score]

  return similarities

In [21]:
#find the best match using keyword similarity (here Jaccard) to find the best matches the input can belong to
def get_code(user_input):

    matched_chapter = match_to_chapter(user_input)
    matched_embeddings=encode_text_matched(matched_chapter, 15)

    input_encoded=encode_input(user_input)
    similarities= find_similar(matched_embeddings, input_encoded)
    sorted_similarities = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
    top_5_similar_hs_codes = sorted_similarities[:5]
    print(top_5_similar_hs_codes)



In [28]:
get_code("poultry")

[('2089010', ['meat edible meat offal fresh chilled frozen wild animals', 0.64313185]), ('2084000', ['meat edible meat offal fresh chilled frozen whales dolphins porpoises mammals order Cetacea manatees dugongs mammals order Sirenia seals sea lions walruses mammals sub order Pinnipedia', 0.6672871]), ('2085000', ['meat edible meat offal fresh chilled frozen reptiles including snakes turtles', 0.6645506]), ('2081000', ['meat edible meat offal fresh chilled frozen rabbits hares', 0.5784572]), ('2083000', ['meat edible meat offal fresh chilled frozen primates', 0.5850519])]
